# Autoencoder를 이용한 시계열 이상 감지

이 스크립트는 재구성 컨볼루션 오토인코더(reconstruction convolutional autoencoder) 모델을 사용하여 시계열 데이터의 이상(anomaly)을 감지하는 방법을 보여줍니다.

## Load the data

[Numenta Anomaly Benchmark(NAB)](
https://www.kaggle.com/boltzmannbrain/nab) (Dataset and scoring for detecting anomalies in streaming data) 데이터 세트를 사용합니다. 이 데이터세트는 레이블이 지정된 비정상적인 동작 기간이 포함된 인공 시계열 데이터를 제공합니다. 데이터는 순서가 있는 타임스탬프가 지정된 단일 값 메트릭입니다.

훈련에는 `anomaly가 없는` `art_daily_small_noise.csv` 파일을 사용하고 테스트에는 `인공 anomaly가 있는` `art_daily_jumpsup.csv` 파일을 사용합니다. 이 데이터 세트의 단순성 덕분에 이상 탐지를 효과적으로 시연할 수 있습니다.

## data 시각화
### 이상이 없는 시계열 데이터
훈련을 위해 다음 데이터를 사용할 것입니다.

### 이상이 있는 시계열 데이터
테스트를 위해 다음 데이터를 사용하고 데이터의 갑작스러운 점프가 이상으로 감지되는지 확인합니다.

## 훈련 데이터 준비

훈련 시계열 데이터 파일에서 데이터 값을 가져오고 '값' 데이터를 정규화합니다. 14일 동안 5분마다 '값'이 있습니다.

-   24 * 60 / 5 = **288 timesteps per day**
-   288 * 14 = total **4032 data points** 

### 시퀀스 생성
훈련 데이터에서 'TIME_STEPS' 연속 데이터 값을 결합하여 시퀀스를 생성했습니다.

## Build a model

convolutional reconstruction autoencoder model을 구축합니다. 모델은 `(batch_size, sequence_length, num_features)` shape의 입력을 받고 동일한 shape의 출력을 반환합니다. 이 경우 `sequence_length`는 288이고 `num_features`는 1입니다.

전치 합성곱층(convolutional transpose) 사용

### 인코더와 디코더 연결하기
- autoencoder의 입력은 encoder의 입력과 동일  
- autoencoder의 출력은 decoder의 출력. decoder는 encoder의 출력을 입력으로 받는다.  

## Train the model

재구성 모델이기 때문에 입력과 대상 모두로 `x_train`을 사용하고 있습니다.

훈련이 어떻게 진행되었는지 보기 위해 훈련 및 검증 손실을 플롯해 보겠습니다.

## Detecting anomalies

모델이 입력 데이터를 얼마나 잘 재구성할 수 있는지 결정하여 이상을 감지합니다.

1. 훈련 샘플에서 MAE loss를 찾습니다.
2. 최대 MAE loss 값을 찾습니다. 이것은 우리 모델이 샘플을 재구성하기 위해 수행한 최악의 작업입니다. 우리는 이것을 이상 감지에 대한 '임계값'으로 정합니다.  
3. 샘플의 재구성 손실이 이 '임계값' 값보다 크면 모델이 익숙하지 않은 패턴을 보고 있다고 추론할 수 있습니다. 우리는 이 샘플을 '이상(anomaly)'으로 분류할 것입니다.


### Compare recontruction

우리 모델이 첫 번째 샘플을 어떻게 재구성했는지 시각화해 봅니다.
이것은 train dataset day 1 으로 부터  288개 timestep 입니다.

### Prepare test data

## Plot anomalies

이제 우리는 비정상 데이터 샘플을 알고 있습니다. 이를 통해 원본 테스트 데이터에서 해당 `Timestamp`를 찾을 수 있습니다. 이를 위해 다음 방법을 사용할 것입니다.

time_steps = 3이고 10개의 훈련 값이 있다고 가정해 보겠습니다. 우리의 `X_train`은 다음과 같이 보일 것입니다:

- 0, 1, 2
- 1, 2, 3
- 2, 3, 4
- 3, 4, 5
- 4, 5, 6
- 5, 6, 7
- 6, 7, 8
- 7, 8, 9

초기 및 최종 time_steps-1 데이터 값을 제외한 모든 값은 'time_steps' 샘플 수로 표시됩니다. 따라서 샘플 [(3, 4, 5), (4, 5, 6), (5, 6, 7)]이 모두 비정상임을 안다면 데이터 포인트 5가 비정상이라고 말할 수 있습니다.

모든 element가 anomaly인 구간 출력

test dat 플롯에 anomaly detect 된 부분을 overlay해 보겠습니다.

이상치 부분을 제대로 detect 한 것을 확인하였습니다.